# Intentional homicide rates in EU countries in 2016

In [1]:
import os
import folium
import pandas as pd
import numpy as np
from pyjstat import pyjstat

In [2]:
m = folium.Map(location=[51.0295293,13.6980529], tiles='Mapbox Bright',zoom_start=4)


In [3]:
import urllib.request, json 
# For some reason, folium is only able to read 4326 projected geojson
URL_GEOJSON = "https://ec.europa.eu/eurostat/cache/GISCO/distribution/v2/nuts/geojson/NUTS_RG_20M_2016_4326_LEVL_0.geojson"
#URL_GEOJSON = "https://raw.githubusercontent.com/eurostat/Nuts2json/gh-pages/2016/4258/20M/nutsbn_1.json"
with urllib.request.urlopen(URL_GEOJSON) as url:
    geo_json_data = json.loads(url.read().decode())
    

In [4]:
folium.GeoJson(geo_json_data).add_to(m)

#m

In [5]:
dataset_code = "crim_off_cat"
query = dataset_code + "?precision=1&unit=P_HTHAB&time=2016"
# Read the data
DATA_URL="http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/{:}".format(query)
jstatfile = dataset_code + ".jstat"

newdata = True
if os.path.exists(jstatfile) and not newdata:
    print("Data retrieved from {:}.".format(jstatfile))
    with open(jstatfile, "r") as f:
        dataset = pyjstat.Dataset.read(f.read())
else:
    dataset = pyjstat.Dataset.read(DATA_URL)
    with open(jstatfile, "w") as f:
        f.write(dataset.write())

df = dataset.write('dataframe')
dfbck = df # backup

In [6]:
# Filters
df = df[df.iccs.str.contains("Intentional homicide")]


In [7]:
# Prepare the data
# Missing data is replaced with the latest avaliable 
idx = df[df.geo == "Netherlands"].index[0]
df.iat[idx,4] = 0.74 # 2013 (Eurostat)
idx = df[df.geo == "Belgium"].index[0]
df.iat[idx,4] = 1.96 # 2015 (Eurostat)
idx = df[df.geo == "North Macedonia"].index[0]
df.iat[idx,4] = 1.21 # 2014 (Eurostat)
idx = df[df.geo == "Turkey"].index[0]
df.iat[idx,4] = 2.42 # 2012 (Eurostat)
#df

In [8]:
nuts_name = dataset["dimension"]["geo"]["category"]["label"]
name_nuts = {v: k for k, v in nuts_name.items()}


# Convert to country level
#UK = England + Wales + Scotland + North Ireland
name_nuts["United Kingdom"] = "UK"
s = pd.Series([df.iccs.iloc[0],df.unit.iloc[0],"United Kingdom",df.time.iloc[0],
               np.average([df[df.geo == 'England and Wales'].iloc[0]['value'],
                           df[df.geo == 'Northern Ireland (UK)'].iloc[0]['value'],
                           df[df.geo == 'Scotland'].iloc[0]['value']])    ],
             index = df.columns)
#               3.], index=df.columns)
df = df.append(s, ignore_index=True)
df[df.geo == "United Kingdom"]

,iccs,unit,geo,time,value
41,Intentional homicide,Per hundred thousand inhabitants,United Kingdom,2016,1.106667


In [9]:
cntry_codes = []
for i in df.geo:
    cntry_codes.append(name_nuts[i])

df["cntry"] = cntry_codes

In [10]:
#df.head(30)

In [11]:
folium.Choropleth(
    geo_data=geo_json_data,
    name='Intentional homicides',
    data=df,
    columns=['cntry','value'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Intentional homicides per 100 000 person (2016)',
).add_to(m)


folium.LayerControl('topleft').add_to(m)

In [12]:
#m

In [13]:
m.save("index.html")

In [14]:
df = df.fillna(0)
highest_rate = df.value.sort_values().tail(8).index


In [15]:
df.iloc[highest_rate[::-1]]

,iccs,unit,geo,time,value,cntry
23,Intentional homicide,Per hundred thousand inhabitants,Latvia,2016,5.64,LV
21,Intentional homicide,Per hundred thousand inhabitants,Lithuania,2016,4.92,LT
24,Intentional homicide,Per hundred thousand inhabitants,Montenegro,2016,3.86,ME
10,Intentional homicide,Per hundred thousand inhabitants,Estonia,2016,2.51,EE
0,Intentional homicide,Per hundred thousand inhabitants,Albania,2016,2.46,AL
36,Intentional homicide,Per hundred thousand inhabitants,Turkey,2016,2.42,TR
3,Intentional homicide,Per hundred thousand inhabitants,Belgium,2016,1.96,BE
16,Intentional homicide,Per hundred thousand inhabitants,Hungary,2016,1.91,HU


In [16]:
#name_nuts

In [17]:
#df[df.time == '2016']

In [18]:
#geo_json_data

In [20]:
#df

,iccs,unit,geo,time,value,cntry
0,Intentional homicide,Per hundred thousand inhabitants,Albania,2016,2.460000,AL
1,Intentional homicide,Per hundred thousand inhabitants,Austria,2016,0.600000,AT
2,Intentional homicide,Per hundred thousand inhabitants,Bosnia and Herzegovina,2016,0.000000,BA
3,Intentional homicide,Per hundred thousand inhabitants,Belgium,2016,1.960000,BE
4,Intentional homicide,Per hundred thousand inhabitants,Bulgaria,2016,1.130000,BG
5,Intentional homicide,Per hundred thousand inhabitants,Switzerland,2016,0.540000,CH
6,Intentional homicide,Per hundred thousand inhabitants,Cyprus,2016,1.300000,CY
7,Intentional homicide,Per hundred thousand inhabitants,Czechia,2016,0.620000,CZ
8,Intentional homicide,Per hundred thousand inhabitants,Germany (until 1990 former territory of the FRG),2016,0.910000,DE
9,Intentional homicide,Per hundred thousand inhabitants,Denmark,2016,0.860000,DK
